<a href="https://colab.research.google.com/github/sornavel/Hackathons-Janatahack/blob/master/Healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

In [ ]:
import os
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
np.random.seed(0)
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
from IPython.core.display import HTML 
from IPython.display import Image
import gc
from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1FUu6ZyHdGB_ElcE4ZuwzPAh-DbQ7mAwQ'})
download.GetContentFile('train.csv')
train_df = pd.read_csv('train.csv')

download = drive.CreateFile({'id': '1vE_M60f3HJ4R1GSFl3QjHWzuwEIcVrmd'})
download.GetContentFile('Third_Health_Camp_Attended.csv')
third_df = pd.read_csv('Third_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '1hnoPLF_uC4hrpCcSFmGJ1UxLog7grgrl'})
download.GetContentFile('Second_Health_Camp_Attended.csv')
second_df = pd.read_csv('Second_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '1LMzFdyra4gObQctf5h0lb6DtUnqiLBT3'})
download.GetContentFile('First_Health_Camp_Attended.csv')
first_df = pd.read_csv('First_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '16JMb9xNz5BEMlV9dS7reh9yi8ivGHYT2'})
download.GetContentFile('Patient_Profile.csv')
patient_df = pd.read_csv('Patient_Profile.csv')

download = drive.CreateFile({'id': '16AFNM3SlDDRMpEAdq4ssddK5QwoH9S33'})
download.GetContentFile('Health_Camp_Detail.csv')
camp_df = pd.read_csv('Health_Camp_Detail.csv')

download = drive.CreateFile({'id': '1aQqKgHFJ0M5j-V34G_TZgOIi2xc6GnaA'})
download.GetContentFile('test.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
first_df.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [ ]:
second_df.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [ ]:
third_df.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [ ]:
patient_df.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [ ]:
camp_df.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [ ]:
test_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [ ]:
third_df_filtered = third_df[third_df.Number_of_stall_visited!=0]

In [ ]:
df = pd.concat([train_df,test_df],axis=0)
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
temp = pd.merge(df,first_df,how='left', on=['Patient_ID','Health_Camp_ID'])
temp.Health_Score.loc[~temp.Health_Score.isnull()] = 1
temp.Health_Score.loc[temp.Health_Score.isnull()] = 0
df = temp.rename(columns={'Health_Score':'succ_1'})

temp = pd.merge(df,second_df,how='left', on=['Patient_ID','Health_Camp_ID'])
temp['Health Score'].loc[~temp['Health Score'].isnull()] = 1
temp['Health Score'].loc[temp['Health Score'].isnull()] = 0
df = temp.rename(columns={'Health Score':'succ_2'})

temp = pd.merge(df,third_df_filtered,how='left', on=['Patient_ID','Health_Camp_ID'])
temp['Number_of_stall_visited'].loc[~temp['Number_of_stall_visited'].isnull()] = 1
temp['Number_of_stall_visited'].loc[temp['Number_of_stall_visited'].isnull()] = 0
df = temp.rename(columns={'Number_of_stall_visited':'succ_3'})
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,succ_1,Unnamed: 4,succ_2,succ_3,Last_Stall_Visited_Number
0,489652,6578,10-Sep-05,4,0,0,0,2,NaN,0.0,NaN,0.0,1.0,1.0
1,507246,6578,18-Aug-05,45,5,0,0,7,NaN,0.0,NaN,0.0,0.0,NaN
2,523729,6534,29-Apr-06,0,0,0,0,0,NaN,0.0,NaN,1.0,0.0,NaN
3,524931,6535,07-Feb-04,0,0,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
4,521364,6529,28-Feb-06,15,1,0,0,7,NaN,0.0,NaN,1.0,0.0,NaN


In [ ]:
df.drop(['Donation','Unnamed: 4','Last_Stall_Visited_Number'],axis=1,inplace=True)

In [ ]:
df['Reg_year'] = pd.DatetimeIndex(df.Registration_Date).year
df['Reg_month'] = pd.DatetimeIndex(df.Registration_Date).month
df['Reg_day'] = pd.DatetimeIndex(df.Registration_Date).day

In [ ]:
temp = pd.merge(df,patient_df,on='Patient_ID',how='left')
df=temp
temp = pd.merge(df,camp_df,on='Health_Camp_ID',how='left')
df=temp

In [ ]:
df.drop(['Patient_ID','Health_Camp_ID','Registration_Date',],axis=1,inplace=True)

In [ ]:
df.Income[df.Income=='None'] = 0
df.Education_Score[df.Education_Score=='None'] = 0
df.Age[df.Age=='None'] = 0
df.Age = df.Age.astype('int64')
df.Age[df.Age==0] = df.Age[df.Age!=0].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df['fi_year'] = pd.DatetimeIndex(df.First_Interaction).year
df['fi_month'] = pd.DatetimeIndex(df.First_Interaction).month
df['fi_day'] = pd.DatetimeIndex(df.First_Interaction).day

In [ ]:
df.drop(['First_Interaction'],axis=1,inplace=True)

In [ ]:
#temp = df.City_Type[~df.City_Type.isnull()].mode()
df.City_Type = df.City_Type.replace(np.nan,'B')
df.Employer_Category = df.Employer_Category.replace(np.nan,'Technology')

In [ ]:
temp = pd.to_datetime(df.Camp_End_Date) - pd.to_datetime(df.Camp_Start_Date)
df['camp_days'] = temp.dt.days

In [ ]:
df.drop(['Camp_Start_Date', 'Camp_End_Date'],axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
df['City_Type']= le.fit_transform(df['City_Type']) 
df['Category1']= le.fit_transform(df['Category1']) 
df['Category2']= le.fit_transform(df['Category2']) 
df['Employer_Category']= le.fit_transform(df['Employer_Category']) 


In [ ]:
#temp = df.Reg_year[~df.Reg_year.isnull()].mode() = 2006
df.Reg_year = df.Reg_year.replace(np.nan,2006)
df.Reg_month = df.Reg_month.replace(np.nan,9)
df.Reg_day = df.Reg_day.replace(np.nan,18)

In [ ]:
df['succ'] = df['succ_1'] + df['succ_2'] + df['succ_3']

In [ ]:
y_train = df['succ'][:train_df.shape[0]]
df.drop(['succ','succ_1','succ_2','succ_3'],axis=1,inplace = True)

In [ ]:
df.head()

,Var1,Var2,Var3,Var4,Var5,Reg_year,Reg_month,Reg_day,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,fi_year,fi_month,fi_day,camp_days
0,4,0,0,0,2,2005.0,9.0,10.0,0,0,0,0,0,0,48.20876,1,11,2,6,2,2004,12,6,59
1,45,5,0,0,7,2005.0,8.0,18.0,0,0,0,0,1,75,40.00000,2,7,2,6,2,2004,9,8,59
2,0,0,0,0,0,2006.0,4.0,29.0,0,0,0,0,0,0,48.20876,1,11,1,0,2,2004,6,22,751
3,0,0,0,0,0,2004.0,2.0,7.0,0,0,0,0,0,0,48.20876,8,11,0,4,2,2004,2,7,17
4,15,1,0,0,7,2006.0,2.0,28.0,0,0,0,1,1,70,40.00000,8,11,1,0,2,2003,7,4,4


In [ ]:
!pip install catboost
!pip install category_encoders
train_cols=df.columns
train = df[:train_df.shape[0]]
test = df[train_df.shape[0]:]

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from catboost import Pool, CatBoostClassifier
from category_encoders import TargetEncoder
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model', n_folds=5):
    kf = StratifiedKFold(n_splits=n_folds, shuffle = True, random_state = 228)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    feature_importances = pd.DataFrame()
    feature_importances['feature'] = test.columns
    i = 1
    for dev_index, val_index in fold_splits:
        print('-------------------------------------------')
        print('Started ' + label + ' fold ' + str(i) + f'/{n_folds}')
        dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
        dev_y, val_y = target.iloc[dev_index], target.iloc[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y, fi = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        feature_importances[f'fold_{i}'] = fi
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score), '\n')
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std score : {}'.format(label, np.std(cv_scores)))
    pred_full_test = pred_full_test / n_folds
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores, 'fi': feature_importances}
    return results

In [ ]:
train.shape, test.shape

((75278, 24), (35249, 24))

In [ ]:
def runCAT(train_X, train_y, test_X, test_y, test_X2, params):
    # Pool the data and specify the categorical feature indices
    print('Pool Data')
    _train = Pool(train_X, label=train_y)
    _valid = Pool(test_X, label=test_y)    
    print('Train CAT')
    model = CatBoostClassifier(**params)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=1000,
                          plot=False)
    feature_im = fit_model.feature_importances_
    print('Predict 1/2')
#    pred_test_y = logit(fit_model.predict_proba(test_X)[:, 1])
    pred_test_y = fit_model.predict_proba(test_X)[:, 1]
    print('Predict 2/2')
    pred_test_y2 = fit_model.predict_proba(test_X2)[:, 1]
#    pred_test_y2 = logit(fit_model.predict_proba(test_X2)[:, 1])    
    return pred_test_y, pred_test_y2, feature_im


# Use some baseline parameters
cat_params = {'loss_function': 'CrossEntropy', 
              'eval_metric': "AUC",
              'learning_rate': 0.01,
              'iterations': 10000,
              'random_seed': 42,
              'od_type': "Iter",
              'early_stopping_rounds': 150,
             }

In [ ]:
from sklearn.model_selection import StratifiedKFold
from scipy.special import logit

n_folds = 10
results = run_cv_model(train[train_cols], test[train_cols], y_train, runCAT, cat_params, auc, 'cat', n_folds=n_folds)
day = 4
sub = 2

-------------------------------------------
Started cat fold 1/10
Pool Data
Train CAT
0:	test: 0.8023226	best: 0.8023226 (0)	total: 32.9ms	remaining: 5m 28s
1000:	test: 0.8502603	best: 0.8502603 (1000)	total: 33.2s	remaining: 4m 58s
2000:	test: 0.8585605	best: 0.8585605 (2000)	total: 1m 5s	remaining: 4m 23s
3000:	test: 0.8623994	best: 0.8623994 (3000)	total: 1m 38s	remaining: 3m 49s
4000:	test: 0.8644826	best: 0.8644858 (3999)	total: 2m 11s	remaining: 3m 17s
5000:	test: 0.8658705	best: 0.8658705 (5000)	total: 2m 44s	remaining: 2m 44s
6000:	test: 0.8668803	best: 0.8668809 (5996)	total: 3m 17s	remaining: 2m 11s
7000:	test: 0.8677500	best: 0.8677553 (6986)	total: 3m 49s	remaining: 1m 38s
8000:	test: 0.8684705	best: 0.8684965 (7979)	total: 4m 22s	remaining: 1m 5s
9000:	test: 0.8690414	best: 0.8690490 (8998)	total: 4m 55s	remaining: 32.7s
9999:	test: 0.8693538	best: 0.8693648 (9935)	total: 5m 27s	remaining: 0us

bestTest = 0.8693648367
bestIteration = 9935

Shrink model to first 9936 iterat

In [ ]:
from math import exp
sub_df = pd.DataFrame(test_df)
sub_df["Outcome"] = results['test']
sub_final_df = test_df[['Patient_ID','Health_Camp_ID','Outcome']]
#sub_df.redemption_status = sub_df.redemption_status.apply(lambda x : 1 if x > 0.1 else 0)
sub_final_df.to_csv("submission.csv", index=False)
sub_final_df.head()
#sub_df.sort_values('redemption_status',ascending=False)

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.676145
1,500633,6584,0.481240
2,506945,6582,0.077251
3,497447,6551,0.863161
4,496446,6533,0.052840


In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>